In [ ]:
from typing import Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import re
from datasets import load_dataset
from nltk.tokenize import ToktokTokenizer
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

## Загрузите эмбеддинги слов
Реализуйте функцию по загрузке эмбеддингов из файла. Она должна отдавать словарь слов и `np.array`
Формат словаря:
```python
{
    'aabra': 0,
    ...,
    'mom': 6546,
    ...
    'xyz': 100355
}
```
Формат матрицы эмбеддингов:
```python
array([[0.44442278, 0.28644582, 0.04357426, ..., 0.9425766 , 0.02024289,
        0.88456545],
       [0.77599317, 0.35188237, 0.54801261, ..., 0.91134102, 0.88599103,
        0.88068835],
       [0.68071886, 0.29352313, 0.95952505, ..., 0.19127958, 0.97723054,
        0.36294011],
       ...,
       [0.03589378, 0.85429694, 0.33437761, ..., 0.39784873, 0.80368014,
        0.76368042],
       [0.01498725, 0.78155695, 0.80372969, ..., 0.82051826, 0.42314861,
        0.18655465],
       [0.69263802, 0.82090775, 0.27150426, ..., 0.86582747, 0.40896573,
        0.33423976]])
```

Количество строк в матрице эмбеддингов должно совпадать с размером словаря, то есть для каждого токена должен быть свой эмбеддинг. По параметру `num_tokens` должно брать не более указано в этом параметре количество токенов в словарь и матрицу эмбеддингов.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.vec

In [ ]:
def load_embeddings(path, num_tokens=100_000):
    """
    load_embeddings
    """
    token2index: Dict[str, int] = {
        "PAD":0,
        "UNK":1
    }
    with open(path, "r") as file:
        vocab_size, emb_dim = file.readline().strip().split()
        vocab_size, emb_dim = (int(vocab_size), int(emb_dim))
        num_tokens = min(num_tokens, vocab_size)
        embeddings = [
            np.zeros(emb_dim),
            np.ones(emb_dim)
        ]
        for line in file:
            parts = line.strip().split()
            token = " ".join(parts[:emb_dim]).lower()
            embedding = np.array(list(map(float, parts[-embedding_dim:])))
            if token in token2index:
                continue
            token2index[token] = len(token2index)
            embeddings.append(embedding)
            if len(token2index) > num_tokens:
                break
    embeddings_matrix: np.array = np.array(embeddings)
    # Необязательно задавать здесь
    # Это рекомендация к типу
    
    assert(len(token2index) == embeddings_matrix.shape[0])
    
    return token2index, embeddings_matrix

token2index, embeddings = load_embeddings("wiki.en.vec")

## Загружаем данные из библиотеки
Мы сразу получим `torch.utils.data.Dataset`, который сможем передать в `torch.utils.data.DataLoader`

In [ ]:
dataset_path = "tweet_eval"
dataset_name = "sentiment"

train_dataset = load_dataset(path=dataset_path, name=dataset_name, split="train")
valid_dataset = load_dataset(path=dataset_path, name=dataset_name, split="validation")
test_dataset = load_dataset(path=dataset_path, name=dataset_name, split="test")

Reusing dataset tweet_eval (/Users/a19415907/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
Reusing dataset tweet_eval (/Users/a19415907/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)
Reusing dataset tweet_eval (/Users/a19415907/.cache/huggingface/datasets/tweet_eval/sentiment/1.1.0/12aee5282b8784f3e95459466db4cdf45c6bf49719c25cdb0743d71ed0410343)


In [ ]:

class Dataset(torch.utils.data.Dataset):
    def __init__(self,
                 texts:List[str],
                 targets:List[int],                 
                 word2id:Dict[str]=token2index,
                 MAX_LEN:int=64):
        super().__init__()
        self.text = [torch.LongTensor([word2id[w] if w in word2id else 1 for w in self.preprocess(t)][:MAX_LEN]) for t in texts]
        self.text = torch.nn.utils.rnn.pad_sequence(self.text,
                                                    batch_first=True,
                                                    padding_value=word2id["PAD"])
        self.label = torch.LongTensor(targets)
        
        self.word2id = word2id
        self.MAX_LEN = MAX_LEN
        self.length = len(texts)
        
    def __getitem__(self, item):
        ids = self.text[item]
        y = self.label[item]
        return ids, y
    
    def __len__(self):
        return self.length
    
    @staticmethod
    def preprocess(text):
        return re.findall("\w+", text)

In [ ]:
new_train_dataset = Dataset(train_dataset["text"], train_dataset["label"])
new_valid_dataset = Dataset(valid_dataset["text"], valid_dataset["label"])
new_test_dataset = Dataset(test_dataset["text"], test_dataset["label"])

In [ ]:
train_loader = DataLoader(new_train_dataset, batch_size=128, shuffle=True)
valid_loader = DataLoader(new_valid_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(new_test_dataset, batch_size=128, shuffle=False)

In [ ]:
for x, y in train_loader:
    break

In [ ]:
assert(isinstance(x, torch.Tensor))
assert(len(x.size()) == 2)

assert(isinstance(y, torch.Tensor))
assert(len(y.size()) == 1)

# Реализация DAN

На вход модели будут подавать индексы слов

Шаги:
- Переводим индексы слов в эмбеддинги
- Усредняем эмбеддинги
- Пропускаем усредненные эмбеддинги через `Multilayer Perceptron`
    - Нужно реализовать самому
    
Дополнительно:
- Добавьте `nn.Dropout`, `nn.BatchNorm` по вкусу
- Сделайте усреднение с учетом падов
- Используйте эмбеддинги от берта/роберты/тд (когда-нибудь про это будет целый туториал, а пока предлагают вам попробовать сделать это самим)


## До эпохи
- Сделайте списки/словари/другое, чтобы сохранять нужные данные для расчета метрик(и) по всей эпохе для трейна и валидации

## Во время эпохи
- Используйте [`tqdm`](https://github.com/tqdm/tqdm) как прогресс бар, чтобы понимать как проходит ваше обучение
- Логируйте лосс
- Логируйте метрику(ки) по батчу
- Сохраняйте то, что вам нужно, чтобы посчитать метрик(и) на всю эпоху для трейна и валидации

## После эпохи
- Посчитайте метрик(и) на всю эпоху для трейна и валидации

## После обучения
- Провалидируйтесь на тестовом наборе и посмотрите метрики
- Постройте [`classification_report`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
- Постройте графики:
    - [Confusion Matrix](https://scikit-learn.org/stable/modules/model_evaluation.html#confusion-matrix)
    - [Опционально] Распределение вероятностей мажоритарного класса (то есть для какого-то примера мы выбираем такой класс и вероятность этого выбора такая-то) на трейне/тесте/валидации
        - Если класс был выбран верно и если была ошибка
- Подумайте что еще вам будет полезно для того, чтобы ответить на такие вопросы: 
    - Что в моделе можно улучшить?
    - Все ли хорошо с моими данными?
    - Все ли хорошо с валидацией?
    - Не переобучился ли я?
    - Достаточно ли я посмотрел на данные?
    - Нужно ли мне улучшить предобработку данных?
    - Нужно ли поменять токенизацию или эмбеддинги?
    - Нет ли у меня багов в реализации?
    - Какие типичные ошибки у моей модели?
    - Как я могу их исправить?

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Я выбрал метрику NLLLoss
Почему я выбрал эту метрику:  
Потому что она подходит для многоклассовой классификации.

In [ ]:
class DeepAverageNetwork(nn.Module):
    def __init__(self,
                 word2id:Dict[str] = token2index,
                 embedding_dim = 300,
                 output_dim:int = 3,
                 embeds:torch.Tensor = embeddings):
        
        self.EMB_DIM = embedding_dim
        self.VOCAB_SIZE = len(word2id)
        self.OUT_DIM = output_dim

        self.embeds = nn.Embedding(self.VOCAB_SIZE, self.EMB_DIM).from_pretrained(embeds)
        self.linear1 = nn.Linear(self.EMB_DIM, self.EMB_DIM)
        self.linear2 = nn.Linear(self.EMB_DIM, self.EMB_DIM)
        self.hidden = nn.Linear(self.EMB_DIM, self.OUT_DIM)
        self.relu = nn.LeakuReLU()
        self.act = nn.LogSoftmax(1)

    def forward(self, text):
        embedded = self.embedding(text)
        print(embedded.size())
        mean = torch.mean(embedded, dim=0).float()
        print(mean.size())
        linear1 = self.relu(self.linear1(mean))
        linear2 = self.relu(self.linear2(linear1))
        hidden = self.hidden(linear2)
        return self.act(hidden)

In [ ]:
model = DeepAverageNetwork(output_dim=3,
                           embeds=embeddings)

## Задайте функцию потерь и оптимизатор

In [ ]:
#loss = nn.CrossEntropyLoss()
loss = nn.NLLLoss.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
model = model.to(device)

## Сделайте цикл обучения

In [ ]:
def train(model:nn.Module,
          iterator:torch.utils.data.DataLoader,
          optimizer:torch.optim.Optimizer,
          loss_fn:nn.modules.loss._Loss,
          print_every:int=1000):
    
    epoch_loss:List[float] = []
    epoch_f1:List[float] = []
    model.train()
    
    for i, (texts, ys) in enumerate(iterator):
        optimizer.zero_grad()
        predictions = model(texts.to(device)).squeeze()
        loss = loss_fn(predictions, ys.to(device))
        
        loss.backward()
        optimizer.step()
        preds = predictions.detach().to('cpu').numpy().argmax(1).tolist()
        y_true = ys.tolist()
        
        epoch_loss.append(loss.item())
        epoch_f1.append(f1_score(y_true, preds, average="micro"))
        
        if not (i + 1) % print_every:
            print(f"loss: {np.mean(epoch_loss)}; F1: {np.mean(epoch_f1)}")
    return np.mean(epoch_f1)
    
def evaluate(model:nn.Module,
             iterator:torch.utils.data.DataLoader,
             loss_fn:torch.nn.modules.loss._Loss):
    
    epoch_loss:list = []
    epoch_f1:list = []
        
    model.eval()
    with torch.no_grad():
        for texts, ys in iterator:
            predictions = model(texts.to(device)).squeeze()
            loss = loss_fn(predictions, ys.to(device))
            preds = predictions.detach().to("cpu").numpy().argmax(1).tolist()
            y_true = ys.tolist()
            
            epoch_loss.append(loss.item())
            epoch_f1.append(f1_score(y_true, preds, average="micro"))
    return np.mean(epoch_f1)

In [ ]:
f1s = []
f1s_eval = []

NUM_EPOCHS = 12  

for n_epoch in tqdm(range(NUM_EPOCHS)):
    print(f"Epoch #{str(n_epoch + 1)}:")
    f1s.append(train(model, training_generator, optimizer, criterion))
    ev = evaluate(model, valid_generator, criterion)
    print("Mean F1 score: ", ev)
    f1s_eval.append(ev)

Ellipsis

# Выводы
Напишите небольшой отчет о проделанной работе. Что удалось, в чем не уверены, что делать дальше.